In [1]:
#
import numpy as np

PEMS03_path = './dataset/PEMS/PEMS03.npz'

origin_data = np.load(PEMS03_path)

In [2]:
origin_data['data'].shape

(26208, 358, 1)

In [3]:
train_num = int(origin_data['data'].shape[0]*0.6)

In [4]:
train_num

15724

In [5]:
train_tokens = []
for j in range(200):
    train_tokens.append(origin_data['data'][j:j+108, :, 0])

In [7]:
train_tokens[0].shape

(108, 358)

In [8]:
# itransformer_tcn 학습
from model import iTransformer_TCN 

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from layers.Transformer_EncDec import Encoder, EncoderLayer
from layers.SelfAttention_Family import FullAttention, AttentionLayer
from layers.Embed import DataEmbedding_inverted_TCN

parser = argparse.ArgumentParser(description='iTransformerTCN')


In [22]:

parser.add_argument('--model_id', type=str, default='test', help='model id')
parser.add_argument('--model', type=str, default='iTransformer_TCN',
                        help='model name, options: [iTransformer, iInformer, iReformer, iFlowformer, iFlashformer, iTransformer_TCN]')
parser.add_argument('--data', type=str, default='custom', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/electricity/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='electricity.csv', help='data csv file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')
parser.add_argument('--train_ratio', type=float, default=0.7, help='train data ratio')
parser.add_argument('--test_ratio', type=float, default=0.2, help='test data ratio')
# parser.add_argument('--select_ratio', type=float, default=1.0, help='select data ratio')
parser.add_argument('--two_sided', action='store_true', default=False, help='whether selecting train data as two-sided')
parser.add_argument('--train_step', type=float, default=1.0, help='train data with certain stes. for example train_step=2 means only train even number of data')

# forecasting task
parser.add_argument('--seq_len', type=int, default=96, help='input sequence length')
parser.add_argument('--label_len', type=int, default=48, help='start token length') # no longer needed in inverted Transformers
parser.add_argument('--pred_len', type=int, default=96, help='prediction sequence length')

_StoreAction(option_strings=['--pred_len'], dest='pred_len', nargs=None, const=None, default=96, type=<class 'int'>, choices=None, required=False, help='prediction sequence length', metavar=None)

In [23]:
# iTransformer
parser.add_argument('--exp_name', type=str, required=False, default='MTSF',
                    help='experiemnt name, options:[MTSF, partial_train]')
parser.add_argument('--channel_independence', action='store_true', default=False, help='whether to use channel_independence mechanism')
parser.add_argument('--inverse', action='store_true', help='inverse output data', default=False)
parser.add_argument('--class_strategy', type=str, default='projection', help='projection/average/cls_token')
parser.add_argument('--target_root_path', type=str, default='./data/electricity/', help='root path of the data file')
parser.add_argument('--target_data_path', type=str, default='electricity.csv', help='data file')
parser.add_argument('--efficient_training', action='store_true', default=False, help='whether to use efficient_training (exp_name should be partial train)') # See Figure 8 of our paper for the detail
parser.add_argument('--use_norm', type=int, default=1, help='use norm and denorm')
parser.add_argument('--partial_start_index', type=int, default=0, help='the start index of variates for partial training, '
                                                                       'you can select [partial_start_index, min(enc_in + partial_start_index, N)]')

# tcitransformer
parser.add_argument('--tcn_layers', type=int, default=3, help='number of layers of TCN embedding field')
parser.add_argument('--tcn_kernel_size', type=int, default=2, help='number of kernel size of TCN embedding field')
parser.add_argument('--tcn_dropout', type=float, default=0.1, help='dropout rate of TCN embedding field')
parser.add_argument('--tcn_uniform_layer', action='store_false', help='whether tcn_layer size is uniform', default=True)
parser.add_argument('--tcn_bias', action='store_false', help='Use Bias in TCN Layers', default=True)
parser.add_argument('--augmented_token', action='store_true', help='Use augmented input token with 0 with pred_len instead of usual input token', default=False)

_StoreTrueAction(option_strings=['--augmented_token'], dest='augmented_token', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Use augmented input token with 0 with pred_len instead of usual input token', metavar=None)

In [27]:
tst = ['--pred_len', '12', '--label_len', '12']
args = parser.parse_args(tst)

In [28]:
args

Namespace(is_training=1, model_id='test', model='iTransformer_TCN', data='custom', root_path='./data/electricity/', data_path='electricity.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', train_ratio=0.7, test_ratio=0.2, two_sided=False, train_step=1.0, seq_len=96, label_len=12, pred_len=12, exp_name='MTSF', channel_independence=False, inverse=False, class_strategy='projection', target_root_path='./data/electricity/', target_data_path='electricity.csv', efficient_training=False, use_norm=1, partial_start_index=0, tcn_layers=3, tcn_kernel_size=2, tcn_dropout=0.1, tcn_uniform_layer=True, tcn_bias=True, augmented_token=False)

In [30]:
args.output_attention= None

In [51]:
args.d_model = 512
args.embed = 'timeF'
args.dropout= 0.05
args.e_layers = 3
args.factor = 1
args.n_heads = 16
args.d_ff = 512
args.activation= 'gelu'
args.output_attention=False

In [52]:
model = iTransformer_TCN.Model(args)

DEBUG:: 96 12


In [53]:
model.train()

Model(
  (enc_embedding): DataEmbedding_inverted_TCN(
    (value_embedding): TemporalConvNet(
      (network): Sequential(
        (0): TemporalBlock(
          (conv1): Conv1d(96, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (chomp1): Chomp1d()
          (relu1): ReLU()
          (dropout1): Dropout(p=0.1, inplace=False)
          (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (chomp2): Chomp1d()
          (relu2): ReLU()
          (dropout2): Dropout(p=0.1, inplace=False)
          (net): Sequential(
            (0): Conv1d(96, 512, kernel_size=(2,), stride=(1,), padding=(1,))
            (1): Chomp1d()
            (2): ReLU()
            (3): Dropout(p=0.1, inplace=False)
            (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
            (5): Chomp1d()
            (6): ReLU()
            (7): Dropout(p=0.1, inplace=False)
          )
          (downsample): Conv1d(96, 512, kernel_size=(1,), stride=(1,))
       

In [55]:
args.devices

AttributeError: 'Namespace' object has no attribute 'devices'

In [66]:
batch_x = torch.Tensor([train_tokens[0][:96, :], train_tokens[1][:96, :]] ).float().to('cpu')
batch_y = torch.Tensor([train_tokens[0][72:, :], train_tokens[1][72:, :]] ).float().to('cpu')

/tmp/ipykernel_447075/1947692231.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch_x = torch.Tensor([train_tokens[0][:96, :], train_tokens[1][:96, :]] ).float().to('cpu')


In [69]:
batch_mark_x = None
batch_mark_y = None

In [71]:
res_model = model(batch_x, batch_mark_x, batch_y, batch_mark_y)

In [72]:
res_model.shape

torch.Size([2, 12, 358])

In [81]:
batch_y.shape

torch.Size([2, 36, 358])

In [83]:
batch_y[0][24:,-1]

tensor([ 89., 102., 112., 108., 110., 114., 110., 124., 110., 112., 139., 113.])

In [84]:
batch_x[0, :,-1]

tensor([ 63.,  62.,  57.,  48.,  47.,  51.,  62.,  35.,  42.,  47.,  38.,  31.,
         24.,  37.,  22.,  24.,  40.,  18.,  32.,  27.,  24.,  21.,  22.,  25.,
         30.,  23.,  30.,  28.,  28.,  19.,  20.,  18.,  24.,  27.,  29.,  29.,
         23.,  19.,  20.,  24.,  19.,  24.,  22.,  20.,  16.,  31.,  24.,  35.,
         19.,  28.,  14.,  34.,  29.,  25.,  28.,  26.,  28.,  24.,  28.,  26.,
         30.,  33.,  31.,  37.,  43.,  45.,  46.,  46.,  37.,  46.,  38.,  43.,
         48.,  48.,  60.,  62.,  63.,  71.,  61.,  76.,  53.,  61.,  65.,  59.,
         66.,  61.,  62.,  73.,  80.,  73.,  74.,  69.,  89., 101., 103., 109.])

In [85]:
res_model[0, :, -1]

tensor([42.2433, 42.0601, 32.0897, 53.4160, 49.0016, 19.2137, 33.1631, 50.6903,
        40.6617, 31.9379, 59.1217, 37.0965], grad_fn=<SelectBackward0>)

In [86]:
batch_x[0, :,-1].to_numpy()

AttributeError: 'Tensor' object has no attribute 'to_numpy'